### Multiclass Classification

In [1]:
from tables import *
import pandas as pd
import numpy as np

### Load data and format data

In [2]:
train = pd.read_hdf("train.h5", "train")
test = pd.read_hdf("test.h5", "test")

#train.shape(45324, 101)
train.head(5)

,y,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x91,x92,x93,x94,x95,x96,x97,x98,x99,x100
0,1,0.768117,-0.742052,-0.426554,-0.981865,-0.739627,-0.710880,0.665179,0.857843,-0.621006,...,-0.516481,0.809766,-0.272739,0.853082,0.920256,0.477963,-0.001392,0.792493,-0.993887,0.024454
1,1,0.785173,-0.759261,-0.182494,-0.982210,-0.699956,-0.680122,0.768796,0.833732,-0.677768,...,-0.603206,0.818925,-0.168998,0.898659,0.944296,0.296505,0.055101,0.755631,-0.994284,-0.120421
2,1,0.838828,-0.785715,-0.306292,-0.988036,-0.700962,-0.657455,0.714290,0.879114,-0.706142,...,-0.609176,0.855403,-0.372542,0.869567,0.912317,0.348275,-0.044856,0.680716,-0.996272,0.058355
3,3,0.774104,-0.714600,-0.279582,-0.976956,-0.698350,-0.701044,0.749821,0.876500,-0.641183,...,-0.502083,0.821006,-0.145995,0.896735,0.950754,0.527539,0.089638,0.782050,-0.994627,-0.112656
4,1,0.754017,-0.757367,-0.403357,-0.982875,-0.678298,-0.753136,0.704445,0.875914,-0.667183,...,-0.707570,0.849876,-0.276707,0.863108,0.957781,0.623078,-0.106684,0.702265,-0.995401,0.106640


In [3]:
y_train = train['y']
x_train = train._drop_axis(['y'], axis=1)

#Switch to numpy
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(test)

### Standardize the data to speed up training

In [4]:
num_features = len(x_train[1])
total = 0

#standardize data, so for each x1..x100, do (x-mean)/sd(x) s.t. mean(x) ~= 0
for i in range(num_features):
    x_train[:,i] = (x_train[:,i] - np.mean(x_train[:,i])) / np.std(x_train[:,i])
    x_test[:,i] = (x_test[:,i] - np.mean(x_test[:,i])) / np.std(x_test[:,i])
    total += np.mean(x_train[:,i])


#adding all the means, we still have a very small # close to 0
total

-0.00011446602163711628

### Create model with Keras
* Iterate through different regularization parameters (L2 norm)
* Output predictions for each lambda into file denoted output_lambda

In [8]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.regularizers import l2

#shortened array to make the notebook shorter
#.005 performed the best, add more values to this array to test different regularization parameters
lambdas = [.005]

for l in lambdas:
    
    #Define model
    model = Sequential([Dense(70, input_shape=(100,), W_regularizer=l2(l)), 
                       Activation('relu'),
                       Dense(30),
                       Activation('relu'),
                       Dense(50),
                       Activation('relu'),
                       Dense(20),
                       Activation('relu'),
                       Dense(5),
                       Activation('softmax'),
    ])
    
    model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

    #Easy way to convert to one-hot
    y_train_hot2 = keras.utils.to_categorical(y_train, num_classes=5)

    model.fit(x_train, y_train_hot2, epochs=50)
    
    pred_one_hot = model.predict(x_test)
    
    pred = []

    for line in pred_one_hot:
        pred.append(np.argmax(line))
        
    d = {'Id': test.index, 'y': pred}
    out = pd.DataFrame(d)
    out.to_csv('output_' + str(l) + '.csv', index=False)
    

c:\python\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(70, input_shape=(100,), kernel_regularizer=<keras.reg...)`
  del sys.path[0]


Epoch 1/50
45324/45324 [==============================] - 2s 35us/step - loss: 0.6856 - acc: 0.8212
Epoch 2/50
45324/45324 [==============================] - 2s 38us/step - loss: 0.4286 - acc: 0.8798
Epoch 3/50
45324/45324 [==============================] - 2s 39us/step - loss: 0.3823 - acc: 0.8903
Epoch 4/50
45324/45324 [==============================] - 2s 35us/step - loss: 0.3594 - acc: 0.8943
Epoch 5/50
45324/45324 [==============================] - 2s 34us/step - loss: 0.3472 - acc: 0.8975: 0s - loss: 0.336
Epoch 6/50
45324/45324 [==============================] - 2s 35us/step - loss: 0.3370 - acc: 0.9020
Epoch 7/50
45324/45324 [==============================] - 2s 33us/step - loss: 0.3312 - acc: 0.9037
Epoch 8/50
45324/45324 [==============================] - 1s 33us/step - loss: 0.3268 - acc: 0.9064
Epoch 9/50
45324/45324 [==============================] - 2s 36us/step - loss: 0.3228 - acc: 0.9078
Epoch 10/50
45324/45324 [==============================] - 2s 44us/step - loss: 0.